Download GNPS data <br>
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" "https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=b817262cb6114e7295fee4f73b22a3ad&view=download_cytoscape_data" -o GNPS_output_graphML_FeatureBased.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2237k    0 2237k    0     0  2237k      0 --:--:-- --:--:-- --:--:-- 4336k
100 3607k    0 3607k    0     0  3607k      0 --:--:-- --:--:-- --:--:-- 4442k


In [2]:
from zipfile import ZipFile
zf = ZipFile("GNPS_output_graphML_FeatureBased.zip", 'r')
zf.extractall("GNPS_output_graphML_FeatureBased")
zf.close()

specify your NAP job ID:

In [3]:
nap_id = 'c4bb6b8be9e14bdebe87c6ef3abe11f6'

load libraries

In [4]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = 'C:\Program Files\ChemAxon\MarvinSuite\bin' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [5]:
if 'clusterinfo_summary' in os.listdir('GNPS_output_graphML_FeatureBased/') and 'DB_result' in os.listdir('GNPS_output_graphML_FeatureBased/'):
    netfile = 'GNPS_output_graphML_FeatureBased/clusterinfo_summary/' + str(os.listdir('GNPS_output_graphML_FeatureBased/clusterinfo_summary/')[0]) 
    gnpslibfile = 'GNPS_output_graphML_FeatureBased/DB_result/'+ str(os.listdir('GNPS_output_graphML_FeatureBased/DB_result/')[0]) 

elif 'clusterinfosummarygroup_attributes_withIDs_withcomponentID' in os.listdir('GNPS_output_graphML_FeatureBased/'):
    netfile = 'GNPS_output_graphML_FeatureBased/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output_graphML_FeatureBased/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
    gnpslibfile = 'GNPS_output_graphML_FeatureBased/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML_FeatureBased/result_specnets_DB/')[0])

else:
    netfile = 'GNPS_output_graphML_FeatureBased/clusterinfosummary/' + str(os.listdir('GNPS_output_graphML_FeatureBased/clusterinfosummary/')[0])
    gnpslibfile = 'GNPS_output_graphML_FeatureBased/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML_FeatureBased/result_specnets_DB/')[0]) 

In [6]:
nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')

add all chemical structural information output as dataframe items in list

In [7]:
matches = [gnpslib, nap]

In [8]:
out = unique_smiles(matches)

In [9]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [10]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys_FeatureBased.txt

'molconvert' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
ikeys = pd.read_csv("InchiKeys_FeatureBased.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys_FeatureBased.txt", quoting=csv.QUOTE_NONE, escapechar='&')

FileNotFoundError: File b'InchiKeys.txt' does not exist

retrieve ClassyFire classifcations

In [ ]:
get_classifications("InchiKeys_FeatureBased.txt")

In [ ]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [ ]:
final.head()

write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [ ]:
final.to_csv("ClassyFireResults_Network_FeatureBased.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [ ]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

which InChIKeys could not be classified?

In [ ]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

create graphML file

In [ ]:
if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML_FeatureBased/' + [x for x in os.listdir('GNPS_output_graphML_FeatureBased/') if 'FEATURE' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML_FeatureBased/')):
    graphMLfile = 'GNPS_output_graphML_FeatureBased/' + [x for x in os.listdir('GNPS_output_graphML_FeatureBased/') if 'METABOLOMICS' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network_FeatureBased.graphml", infer_numeric_types = True)
else:
    print('There is no graphML file for this GNPS molecular network job')